In [1]:
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import cv2

import tensorflow as tf
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from keras.losses import categorical_crossentropy
from keras.optimizers import adam, sgd
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping

from PIL import Image

Using TensorFlow backend.


In [0]:
import os

In [3]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
#go to working directory
cd "/content/drive/My Drive/dataset/"

/content/drive/My Drive/dataset


In [0]:
#define the variables
train_path = 'training_set'
test_path = 'test_set'
IMG_BREDTH = 30
IMG_HEIGHT = 60
num_classes = 2

In [6]:
train_batch = ImageDataGenerator(featurewise_center=False,
                                 samplewise_center=False, 
                                 featurewise_std_normalization=False, 
                                 samplewise_std_normalization=False, 
                                 zca_whitening=False, 
                                 rotation_range=45, 
                                 width_shift_range=0.2, 
                                 height_shift_range=0.2, 
                                 horizontal_flip=True, 
                                 vertical_flip=False).flow_from_directory(train_path, 
                                                                          target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                                          classes=['compostable_wastes', 'recyclable_wastes'], 
                                                                          batch_size=100)

test_batch = ImageDataGenerator().flow_from_directory(test_path, 
                                                      target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                      classes=['compostable_wastes', 'recyclable_wastes'], 
                                                      batch_size=100)

Found 3111 images belonging to 2 classes.
Found 779 images belonging to 2 classes.


In [0]:
batch_size = 10
epochs = 15

In [0]:
def cnn_model():
    
    model = Sequential()

    model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(IMG_HEIGHT,IMG_BREDTH,3)))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(Dropout(0.25))
    
    model.add(Flatten())

    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(num_classes, activation='softmax'))
    
              
    model.summary()
              
    return model

def use_model(path):
    
    model = load_model('best_waste_classifier.h5')
    pic = plt.imread(path)
    pic = cv2.resize(pic, (IMG_BREDTH, IMG_HEIGHT))
    pic = np.expand_dims(pic, axis=0)
    classes = model.predict_classes(pic)
  
    return classes

In [9]:
model = cnn_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 60, 30, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 58, 28, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 26, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 28, 13, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 28, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 26, 11, 64)        18496     
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 24, 9, 64)         36928     
__________

In [10]:
path, data_dir_list, files = next(os.walk(train_path))
data_dir_list

['recyclable_wastes', 'compostable_wastes']

In [11]:
#Calculate Train samples
train_samples = 0

path, dirs, files = next(os.walk(train_path))
nb_classsifires = len(dirs)
print(nb_classsifires)
for d in dirs:
    path, nub_classifires, files = next(os.walk(train_path + "/" + d))
    train_samples = train_samples+len(files)
    
print(train_samples ) 

2
3113


In [12]:
test_samples = 0

path, dirs, files = next(os.walk(test_path))
nb_classsifires = len(dirs)
print(nb_classsifires)
for d in dirs:
    path, nub_classifires, files = next(os.walk(test_path + "/" + d))
    test_samples = test_samples+len(files)
    
print(test_samples) 

2
781


In [0]:
checkpoint = ModelCheckpoint('best_waste_classifier.h5', 
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='auto')

In [0]:
model.compile(loss='categorical_crossentropy', optimizer=adam(lr=1.0e-4), metrics=['accuracy'])
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=10, verbose=1, mode='auto')

In [15]:
# Train the model 
model = model.fit_generator(train_batch,
                          steps_per_epoch = train_samples/50,
                          epochs = epochs,
                          validation_data = test_batch,
                          validation_steps = test_samples,
                          callbacks = [checkpoint, early])

Epoch 1/15
63/62 [==============================] - 808s 13s/step - loss: 0.7599 - acc: 0.6193 - val_loss: 0.5269 - val_acc: 0.7356
Epoch 2/15
63/62 [==============================] - 796s 13s/step - loss: 0.5257 - acc: 0.7710 - val_loss: 0.5488 - val_acc: 0.7240
Epoch 3/15
63/62 [==============================] - 794s 13s/step - loss: 0.4952 - acc: 0.7933 - val_loss: 0.5632 - val_acc: 0.7072
Epoch 4/15
63/62 [==============================] - 786s 12s/step - loss: 0.4905 - acc: 0.7987 - val_loss: 0.5398 - val_acc: 0.7639
Epoch 5/15
63/62 [==============================] - 783s 12s/step - loss: 0.4700 - acc: 0.8050 - val_loss: 0.5519 - val_acc: 0.7240
Epoch 6/15
63/62 [==============================] - 783s 12s/step - loss: 0.4605 - acc: 0.8106 - val_loss: 0.5830 - val_acc: 0.7138
Epoch 7/15
63/62 [==============================] - 783s 12s/step - loss: 0.4471 - acc: 0.8160 - val_loss: 0.4984 - val_acc: 0.7766
Epoch 8/15
63/62 [==============================] - 781s 12s/step - loss: 0.